In [156]:
import pandas as pd
import numpy as np

from scipy.optimize import curve_fit
from scipy.special import factorial

import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots

## Obtain and save data from the web

In [2]:
# # the service URL
# livechart = "https://nds.iaea.org/relnsd/v1/data?"

# # There have been cases in which the service returns an HTTP Error 403: Forbidden
# # use this workaround
# import urllib.request
# def lc_pd_dataframe(url):
#     req = urllib.request.Request(url)
#     req.add_header('User-Agent''','' 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
#     return pd.read_csv(urllib.request.urlopen(req))

In [3]:
# ground_states_df = lc_pd_dataframe(livechart + "fields=ground_states&nuclides=all")
# ground_states_df = ground_states_df[['z', 'n', 'symbol', 'half_life']]
# ground_states_df = ground_states_df[ground_states_df['half_life'] == 'STABLE']
# ground_states_df = ground_states_df.copy()
# ground_states_df['a'] = ground_states_df['z'] + ground_states_df['n']
# ground_states_df['symbol'] = ground_states_df['symbol'].str.lower()
# ground_states_df['name'] = ground_states_df['a'].astype(str).str.cat(ground_states_df['symbol'])
# names = ground_states_df['name'].values

In [4]:
# levels_dfs = [lc_pd_dataframe(livechart + f"fields=levels&nuclides={name}") for name in names]
# levels_dfs =  pd.concat(levels_dfs, ignore_index=True)

# print(levels_dfs.columns)
# print(levels_dfs.head())

In [5]:
# levels_dfs.to_csv("levels.csv", index=False)

## Load saved data

In [6]:
dtypes = {
    'z': 'UInt8',
    'n': 'UInt8',
    'symbol': 'string',
    'idx': 'UInt16',
    'energy_shift': 'category',
    'energy': 'Float64',
    'unc_e': 'Float64',
    'ripl_shift': 'Float64',
    'jp': 'string',
    'jp_order': 'UInt8',
    'half_life': 'string',
    'operator_hl': 'string',
    'unc_hl': 'string',
    'unit_hl': 'category',
    'half_life_sec': 'Float64',
    'unc_hl.1': 'Float64',
    'decay_1': 'string',
    'decay_1_%': 'Float64',
    'unc_1': 'Float64',
    'decay_2': 'string',
    'decay_2_%': 'Float64',
    'unc_2': 'Float64',
    'decay_3': 'string',
    'decay_3_%': 'Float64',
    'unc_3': 'Float64',
    'isospin': 'string',
    'magnetic_dipole': 'Float64',
    'unc_mn': 'Float64',
    'electric_quadrupole': 'Float64',
    'unc_eq': 'Float64',
    'ENSDF_publication_cut-off': 'string',
    'ENSDF_authors': 'string',
    'Extraction_date': 'string'
}
parse_dates = ['ENSDF_publication_cut-off', 'Extraction_date']


df = pd.read_csv('levels.csv', parse_dates=parse_dates, dtype=dtypes)


df.loc[df['half_life'] == 'STABLE', 'half_life'] = 'inf'
df['half_life'] = df['half_life'].astype('Float64')

df['a'] = df['z'] + df['n']

df = df[['symbol', 'a', 'z', 'n', 'idx', 'energy', 'energy_shift', 'ripl_shift', 'jp', 'jp_order']]
df['energy'] = df['energy'] / 1000 #MeV
df['ripl_shift'] = df['ripl_shift'] / 1000 #MeV
#TODO also take into account energy uncertainty

df['beta'] = (df['n'].astype('Float64') - df['z'].astype('Float64')) / df['a'].astype('Float64')

df = df.set_index(['z', 'a'], drop=True)

df.head()

symbol  n  idx  energy energy_shift  ripl_shift    jp  jp_order      beta
z a                                                                          
1 1      H  0    0     0.0          NaN        <NA>  1/2+         1      -1.0
  2      H  1    0     0.0          NaN        <NA>    1+         1       0.0
2 3     He  1    0     0.0          NaN        <NA>  1/2+         1 -0.333333
  4     He  2    0     0.0          NaN        <NA>    0+         1       0.0
  4     He  2    1   20.21          NaN        <NA>    0+         2       0.0

In [7]:
# Filter out shifts
#TODO take into account energy shifts with ripl_shift
#TODO take into account energy shifts without ripl_shift

df = df[df['energy_shift'].isna()]
df = df.drop(['energy_shift', 'ripl_shift'], axis='columns')
df.head()

symbol  n  idx  energy    jp  jp_order      beta
z a                                                 
1 1      H  0    0     0.0  1/2+         1      -1.0
  2      H  1    0     0.0    1+         1       0.0
2 3     He  1    0     0.0  1/2+         1 -0.333333
  4     He  2    0     0.0    0+         1       0.0
  4     He  2    1   20.21    0+         2       0.0

In [8]:
# Filter out not enough levels
# Just removes 1H, 2H, and 3He

df = df.loc[df.groupby(level=df.index.names).size() >= 3]

# Filter out uncertain jp

df = df[df['jp'].str.fullmatch(r'^[0-9]+(/[0-9]+)?[+-]$')]

# Extract j and p

df['j'] = df['jp'].str[:-1]
df['p'] = df['jp'].str[-1]

odd_spins = df['j'].str.contains('/')
even_spins = ~odd_spins

df.loc[odd_spins,'j_float'] = df[odd_spins]['j'].str.split('/').apply(lambda x: float(x[0]) / float(x[1]))
df.loc[even_spins,'j_float'] = df[even_spins]['j'].astype(float)

df['j_evenness'] = pd.Series(data=pd.NA, dtype='boolean')
df.loc[even_spins,'j_evenness'] = (df[even_spins]['j'].str.split('/', expand=True)[0].astype(int) % 2 == 0)
#TODO similar for odd_spins

df['p_bit'] = df['p'].str.fullmatch(r'\-').astype(int)

# Search

## For quadrupole oscillator

### Quadrupole, positive, first band 

In [62]:
def get_osc_energy(n, energy_quantum, shift4):
    harmonics = n * energy_quantum
    shifts = factorial(n) / 2 * shift4
    shifts[0:2] = 0

    return harmonics + shifts


def fit(group, func):

    x = group['quanta']
    y = group['energy']

    # fit
    popt, pcov = curve_fit(func, x, y)

    # prediction
    y_pred = func(x, *popt)

    # r-squared
    ss_res = np.sum((y - y_pred)**2)
    ss_tot = np.sum((y - y.mean())**2)
    r2 = 1 - (ss_res / ss_tot)

    # results
    results = pd.DataFrame({'energy_quantum': popt[0], 'shift4': popt[1], 'r2': r2, 'anharmonicity': popt[1]/popt[0], 'j_float': x, 'quanta': x, 'energy_pred': y_pred})

    # results = pd.Series(list(popt)+[r2], index=['energy_quantum', 'zero_point_energy', 'r2'])
    return  results

In [63]:
quad_pos_first = df[(df['jp_order'] == 1) & df['j_evenness'].fillna(False) & (df['p_bit'] == 0)]
quad_pos_first = quad_pos_first.loc[quad_pos_first.groupby(level=df.index.names).size() >= 3]
quad_pos_first['quanta'] = (quad_pos_first['j_float'].astype(int) // 2)

quad_pos_first_groups = quad_pos_first.groupby(level=df.index.names, as_index=False)

oscillator_fit = quad_pos_first_groups.apply(lambda group: fit(group, get_osc_energy)).droplevel(0)

In [64]:
min_r2 = 0.9

print("Oscillator osc:")
best_oscillator_fit = oscillator_fit[oscillator_fit['r2'] > min_r2]

print(best_oscillator_fit.groupby(by=best_oscillator_fit.index)['r2'].head(1))

Oscillator osc:
z   a  
3   6      0.947661
6   12     1.000000
7   14     0.902845
8   16     0.990910
    18     0.967631
             ...   
80  196    1.000000
    198    0.960481
    200    0.984854
    202    0.999936
    204    0.999741
Name: r2, Length: 138, dtype: float64


In [65]:
#TODO plot r^2 on grid
#TODO change starting point

In [66]:
merged = quad_pos_first.merge(oscillator_fit, how='left', on=['z','a', 'quanta'])
best_osc_merged = merged[merged['r2'] > min_r2]

def get_best_r2(df, min_r2):
    return df[df['r2'] > min_r2]

# merged = merged.merge(rotator_fit, how='left', on=['z','a', 'idx'], suffixes=('_osc', '_rot'))
# lowest_merged = merged.groupby(['z','a']).head(3)
lowest_merged = merged
# best_rot_merged = lowest_merged[lowest_merged['r2_rot'] > min_r2]
# best_rot_merged

In [67]:
best_osc_merged

symbol    n  idx    energy  jp  jp_order      beta  j  p  j_float_x  \
z  a                                                                         
3  6       Li    3    2   3.56288  0+         1       0.0  0  +        0.0   
   6       Li    3    3     4.312  2+         1       0.0  2  +        2.0   
   6       Li    3    9      23.0  4+         1       0.0  4  +        4.0   
6  12       C    6    0       0.0  0+         1       0.0  0  +        0.0   
   12       C    6    1   4.43982  2+         1       0.0  2  +        2.0   
...       ...  ...  ...       ...  ..       ...       ... .. ..        ...   
80 202     Hg  122   34   1.98882  6+         1  0.207921  6  +        6.0   
   204     Hg  124    0       0.0  0+         1  0.215686  0  +        0.0   
   204     Hg  124    1  0.436552  2+         1  0.215686  2  +        2.0   
   204     Hg  124    2   1.12823  4+         1  0.215686  4  +        4.0   
   204     Hg  124   15   2.19101  6+         1  0.215686  6  +        6.0   

        j_evenness  p_bit  quanta  energy_quantum     shift4        r2  \
z  a                                                                     
3  6          True      0       0        4.312000  14.376000  0.947661   
   6          True      0       1        4.312000  14.376000  0.947661   
   6          True      0       2        4.312000  14.376000  0.947661   
6  12         True      0       0        4.439820   4.420360  1.000000   
   12         True      0       1        4.439820   4.420360  1.000000   
...            ...    ...     ...             ...        ...       ...   
80 202        True      0       3        0.447809   0.216047  0.999936   
   204        True      0       0        0.418240   0.310062  0.999741   
   204        True      0       1        0.418240   0.310062  0.999741   
   204        True      0       2        0.418240   0.310062  0.999741   
   204        True      0       3        0.418240   0.310062  0.999741   

        anharmonicity  j_float_y  energy_pred  
z  a                                           
3  6         3.333952          0     0.000000  
   6         3.333952          1     4.312000  
   6         3.333952          2    23.000000  
6  12        0.995617          0     0.000000  
   12        0.995617          1     4.439820  
...               ...        ...          ...  
80 202       0.482454          3     1.991568  
   204       0.741349          0     0.000000  
   204       0.741349          1     0.418240  
   204       0.741349          2     1.146542  
   204       0.741349          3     2.184906  

[1032 rows x 19 columns]

In [68]:
merged

symbol    n  idx    energy   jp  jp_order      beta   j  p  j_float_x  \
z  a                                                                           
3  6       Li    3    2   3.56288   0+         1       0.0   0  +        0.0   
   6       Li    3    3     4.312   2+         1       0.0   2  +        2.0   
   6       Li    3    9      23.0   4+         1       0.0   4  +        4.0   
5  10       B    5    2   1.74005   0+         1       0.0   0  +        0.0   
   10       B    5    4   3.58713   2+         1       0.0   2  +        2.0   
...       ...  ...  ...       ...  ...       ...       ...  .. ..        ...   
82 208     Pb  126   20  4.323946   4+         1  0.211538   4  +        4.0   
   208     Pb  126   23  4.423647   6+         1  0.211538   6  +        6.0   
   208     Pb  126   26  4.610748   8+         1  0.211538   8  +        8.0   
   208     Pb  126   38   4.89523  10+         1  0.211538  10  +       10.0   
   208     Pb  126  184   6.10069  12+         1  0.211538  12  +       12.0   

        j_evenness  p_bit  quanta  energy_quantum     shift4        r2  \
z  a                                                                     
3  6          True      0       0        4.312000  14.376000  0.947661   
   6          True      0       1        4.312000  14.376000  0.947661   
   6          True      0       2        4.312000  14.376000  0.947661   
5  10         True      0       0        3.587130  -1.149360  0.672252   
   10         True      0       1        3.587130  -1.149360  0.672252   
...            ...    ...     ...             ...        ...       ...   
82 208        True      0       2        1.315512  -0.005631  0.404821   
   208        True      0       3        1.315512  -0.005631  0.404821   
   208        True      0       4        1.315512  -0.005631  0.404821   
   208        True      0       5        1.315512  -0.005631  0.404821   
   208        True      0       6        1.315512  -0.005631  0.404821   

        anharmonicity  j_float_y  energy_pred  
z  a                                           
3  6         3.333952          0     0.000000  
   6         3.333952          1     4.312000  
   6         3.333952          2    23.000000  
5  10       -0.320412          0     0.000000  
   10       -0.320412          1     3.587130  
...               ...        ...          ...  
82 208      -0.004280          2     2.625394  
   208      -0.004280          3     3.929644  
   208      -0.004280          4     5.194479  
   208      -0.004280          5     6.239713  
   208      -0.004280          6     5.865987  

[1056 rows x 19 columns]

In [69]:
merged.loc[24,52]

symbol   n  idx    energy   jp  jp_order      beta   j  p  j_float_x  \
z  a                                                                         
24 52     Cr  28    0       0.0   0+         1  0.076923   0  +        0.0   
   52     Cr  28    1  1.434091   2+         1  0.076923   2  +        2.0   
   52     Cr  28    2   2.36963   4+         1  0.076923   4  +        4.0   
   52     Cr  28    6  3.113858   6+         1  0.076923   6  +        6.0   
   52     Cr  28   26   4.75031   8+         1  0.076923   8  +        8.0   
   52     Cr  28  125    7.2379  10+         1  0.076923  10  +       10.0   

       j_evenness  p_bit  quanta  energy_quantum    shift4       r2  \
z  a                                                                  
24 52        True      0       0        1.093661  0.029399  0.99374   
   52        True      0       1        1.093661  0.029399  0.99374   
   52        True      0       2        1.093661  0.029399  0.99374   
   52        True      0       3        1.093661  0.029399  0.99374   
   52        True      0       4        1.093661  0.029399  0.99374   
   52        True      0       5        1.093661  0.029399  0.99374   

       anharmonicity  j_float_y  energy_pred  
z  a                                          
24 52       0.026881          0     0.000000  
   52       0.026881          1     1.093661  
   52       0.026881          2     2.216721  
   52       0.026881          3     3.369180  
   52       0.026881          4     4.727434  
   52       0.026881          5     7.232258

In [70]:
df_to_plot = best_osc_merged
# df_to_plot = merged

fig = px.scatter(df_to_plot.groupby(by=df_to_plot.index).head(1).reset_index(), 
          # x='a', 
          x='n',
          y='z', 
          color='r2',
          title='R² Values for Best Oscillator Fits',
          # labels={'z': 'Number of protons (Z)', 'a': 'Weight (A)', 'r2': 'R² Score'},
          labels={'z': 'Number of protons (Z)', 'n': 'Number of neutrons (N)', 'r2': 'R² Score'},
          color_continuous_scale='RdBu',
          height=600)


magic_numbers_n = [2, 8, 20, 50, 58, 82, 126]
magic_numbers_z = [2, 8, 20, 50, 58, 82]

for i, m in enumerate(magic_numbers_n):
    fig.add_vline(x=m, line_dash="dash", line_color="gray", name="Magic numbers" if i==0 else None)

for i, m in enumerate(magic_numbers_z):
    fig.add_hline(y=m, line_dash="dash", line_color="gray")

fig.show()

In [72]:
df_to_plot = merged

px.scatter(df_to_plot.groupby(by=df_to_plot.index).head(1).reset_index(), 
            x='a', 
            y='r2', 
            color='anharmonicity',
            title='R² Values for All Oscillator Fits',
            labels={'anharmonicity': 'Anharmonicity Ratio', 'a': 'Weight (A)', 'r2': 'R² Score'},
            color_continuous_scale='viridis')

In [73]:
df_to_plot = merged

fig = px.scatter(df_to_plot.groupby(by=df_to_plot.index).head(1).reset_index(), 
          # x='a', 
          x='n',
          y='z', 
          color='r2',
          title='R² Values for All Oscillator Fits',
          # labels={'z': 'Number of protons (Z)', 'a': 'Weight (A)', 'r2': 'R² Score'},
          labels={'z': 'Number of protons (Z)', 'n': 'Number of neutrons (N)', 'r2': 'R² Score'},
          color_continuous_scale=[(0, 'red'), (0.8, 'white'), (1, 'blue')],
          height=600)


magic_numbers_n = [2, 8, 20, 50, 58, 82, 126]
magic_numbers_z = [2, 8, 20, 50, 58, 82]

for i, m in enumerate(magic_numbers_n):
    fig.add_vline(x=m, line_dash="dash", line_color="gray", name="Magic numbers" if i==0 else None)

for i, m in enumerate(magic_numbers_z):
    fig.add_hline(y=m, line_dash="dash", line_color="gray")

fig.show()

In [74]:
merged.loc[24,52]

symbol   n  idx    energy   jp  jp_order      beta   j  p  j_float_x  \
z  a                                                                         
24 52     Cr  28    0       0.0   0+         1  0.076923   0  +        0.0   
   52     Cr  28    1  1.434091   2+         1  0.076923   2  +        2.0   
   52     Cr  28    2   2.36963   4+         1  0.076923   4  +        4.0   
   52     Cr  28    6  3.113858   6+         1  0.076923   6  +        6.0   
   52     Cr  28   26   4.75031   8+         1  0.076923   8  +        8.0   
   52     Cr  28  125    7.2379  10+         1  0.076923  10  +       10.0   

       j_evenness  p_bit  quanta  energy_quantum    shift4       r2  \
z  a                                                                  
24 52        True      0       0        1.093661  0.029399  0.99374   
   52        True      0       1        1.093661  0.029399  0.99374   
   52        True      0       2        1.093661  0.029399  0.99374   
   52        True      0       3        1.093661  0.029399  0.99374   
   52        True      0       4        1.093661  0.029399  0.99374   
   52        True      0       5        1.093661  0.029399  0.99374   

       anharmonicity  j_float_y  energy_pred  
z  a                                          
24 52       0.026881          0     0.000000  
   52       0.026881          1     1.093661  
   52       0.026881          2     2.216721  
   52       0.026881          3     3.369180  
   52       0.026881          4     4.727434  
   52       0.026881          5     7.232258

In [75]:
test_nucleus = merged.loc[24,52]
a = test_nucleus.index[0][1]
symbol = test_nucleus.iloc[0]['symbol']

px.scatter(x=test_nucleus['quanta'], 
            y=test_nucleus['energy']-test_nucleus['energy_pred'],
            title=f'Residuals for {a}{symbol}',
            labels={'x': 'Quanta of oscillation', 'y': 'Residuals'})

In [154]:
residuals_df = best_osc_merged.copy()
residuals_df = residuals_df.reset_index()
residuals_df["name"] = residuals_df["a"].astype(str) + residuals_df["symbol"]
residuals_df["residuals"] = residuals_df["energy"] - residuals_df["energy_pred"]

best_residuals = get_best_r2(residuals_df, 0.999)

fig = px.scatter(best_residuals, x="quanta", y="residuals", color='anharmonicity', facet_col="name", facet_col_wrap=8)
fig.add_hline(y=0, line_width=1, line_dash="dash", line_color="black")
fig.show()

In [167]:
best_residuals = get_best_r2(residuals_df, 0.999)

fig = px.scatter(best_residuals, x="quanta", y="energy", color='anharmonicity', facet_col="name", facet_col_wrap=8)


# fig.add_scatter()
# fig.add_scatter(x=best_residuals["quanta"], y=fit_energy, mode='lines', facet_col="name", facet_col_wrap=8)

# fig.add_trace()


facet_col_wrap=8
group_names = list(best_residuals.groupby('name')['name'].head(1))
ngroups = len(group_names)
nrows = ngroups // facet_col_wrap + 1
ncols =  facet_col_wrap + 1

fig = make_subplots(rows=nrows, cols=ncols, subplot_titles=group_names)

for i in range(ngroups):
    row_idx = i // facet_col_wrap
    col_idx = i % facet_col_wrap
    print(row_idx, col_idx, group_names[i])

    group_name = group_names[i]
    nucleus = best_residuals[best_residuals['name'] == group_name]

    quanta = nucleus['quanta']
    energy = nucleus['energy']
    harmonic_energy = get_osc_energy(quanta, nucleus['energy_quantum'], 0)
    fit_energy = get_osc_energy(quanta, nucleus['energy_quantum'], nucleus['shift4'])

    scatter_energies = go.Scatter(x=quanta, y=energy, mode='markers')
    scatter_harmonics  = go.Scatter(x=quanta, y=harmonic_energy, mode='lines')
    scatter_fit = go.Scatter(x=quanta, y=fit_energy, mode='lines')
    fig.add_trace(scatter_energies, row=row_idx+1, col=col_idx+1)
    fig.add_trace(scatter_harmonics, row=row_idx+1, col=col_idx+1)
    fig.add_trace(scatter_fit, row=row_idx+1, col=col_idx+1)

fig.show()

0 0 12C
0 1 22Ne
0 2 30Si
0 3 36S
0 4 50Ti
0 5 54Cr
0 6 64Zn
0 7 66Zn
1 0 68Zn
1 1 74Ge
1 2 80Se
1 3 82Kr
1 4 86Kr
1 5 88Sr
1 6 94Zr
1 7 94Mo
2 0 98Mo
2 1 114Cd
2 2 120Te
2 3 122Te
2 4 132Xe
2 5 134Ba
2 6 194Pt
2 7 196Hg
3 0 202Hg
3 1 204Hg


In [172]:
best_anharmonics = get_best_r2(residuals_df, 0.999)
best_anharmonics = best_anharmonics[best_anharmonics['anharmonicity'] > 0.5]


facet_col_wrap=4
group_names = list(best_anharmonics.groupby('name')['name'].head(1))
ngroups = len(group_names)
nrows = ngroups // facet_col_wrap + 1
ncols =  facet_col_wrap + 1

fig = make_subplots(rows=nrows, cols=ncols, subplot_titles=group_names)

for i in range(ngroups):
    row_idx = i // facet_col_wrap
    col_idx = i % facet_col_wrap
    print(row_idx, col_idx, group_names[i])

    group_name = group_names[i]
    nucleus = best_anharmonics[best_anharmonics['name'] == group_name]

    quanta = nucleus['quanta']
    energy = nucleus['energy']
    harmonic_energy = get_osc_energy(quanta, nucleus['energy_quantum'], 0)
    fit_energy = get_osc_energy(quanta, nucleus['energy_quantum'], nucleus['shift4'])

    scatter_energies = go.Scatter(x=quanta, y=energy, mode='markers')
    scatter_harmonics  = go.Scatter(x=quanta, y=harmonic_energy, mode='lines')
    scatter_fit = go.Scatter(x=quanta, y=fit_energy, mode='lines')
    fig.add_trace(scatter_energies, row=row_idx+1, col=col_idx+1)
    fig.add_trace(scatter_harmonics, row=row_idx+1, col=col_idx+1)
    fig.add_trace(scatter_fit, row=row_idx+1, col=col_idx+1)

fig.show()

0 0 12C
0 1 22Ne
0 2 80Se
0 3 204Hg


In [82]:
df_to_plot = residuals_df[residuals_df['anharmonicity'].abs() < 0.5]

fig = px.scatter(df_to_plot, 
          x='n',
          y='z', 
          color='anharmonicity',
          title='Anharmonic to Harmonic Ratio for All Oscillator Fits',
          labels={'z': 'Number of protons (Z)', 'n': 'Number of neutrons (N)', 'ratio': 'Anharmonicity Ratio'},
          height=600)


magic_numbers_n = [2, 8, 20, 50, 58, 82, 126]
magic_numbers_z = [2, 8, 20, 50, 58, 82]

for i, m in enumerate(magic_numbers_n):
    fig.add_vline(x=m, line_dash="dash", line_color="gray", name="Magic numbers" if i==0 else None)

for i, m in enumerate(magic_numbers_z):
    fig.add_hline(y=m, line_dash="dash", line_color="gray")

fig.show()

In [85]:
df_to_plot = residuals_df[residuals_df['anharmonicity'].abs() < 0.5]

px.scatter(df_to_plot, 
            x='anharmonicity', 
            y='r2', 
            title='R² Values for All Oscillator Fits as Function of Anharmonicity',
            labels={'anharmonicity': 'Anharmonicity Ratio', 'r2': 'R² Score'},
            color_continuous_scale='viridis')